In [1]:
import os
import pandas as pd 
os.chdir('/mnt/BioHome/jreyna/jreyna/projects/dchallenge')

outdir = 'results/main/pc_hic/2016_javierre/processing/'
os.makedirs(outdir, exist_ok=True)

In [2]:
loops = pd.read_table('results/main/pc_hic/2016_javierre/data/PCHiC_peak_matrix_cutoff5.tsv')

# reformatting dataframe into bedpe format
bedpe_reformat_cols = ['baitChr',
         'baitStart',
         'baitEnd',
         'oeChr',
         'oeStart',
         'oeEnd',
         'Mon',
         'Mac0',
         'Mac1',
         'Mac2',
         'Neu',
         'MK',
         'EP',
         'Ery',
         'FoeT',
         'nCD4',
         'tCD4',
         'aCD4',
         'naCD4',
         'nCD8',
         'tCD8',
         'nB',
         'tB',
         'baitID',
         'baitName',
         'oeID',
         'oeName',
         'dist',
         'clusterID',
         'clusterPostProb']

main_bedpe = loops.loc[:, bedpe_reformat_cols]

/mnt/beegfs/jreyna/pbstmp.5039141.herman.hpc.lji.org/ipykernel_21035/2537193064.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  loops = pd.read_table('results/main/pc_hic/2016_javierre/data/PCHiC_peak_matrix_cutoff5.tsv')


In [3]:
# listing cells within the dataframe
cells = ['Mon', 'Mac0', 'Mac1', 'Mac2',
             'Neu', 'MK', 'EP', 'Ery',
             'FoeT', 'nCD4', 'tCD4', 'aCD4',
             'naCD4', 'nCD8', 'tCD8',
             'nB', 'tB']

# listing the new names from the corresponding cells
new_cells = ['monocytes', 'macrophage0', 'macrophage1', 'macrophage2',
                'neutrophils', 'megakaryocyte', 'epithelial', 'erythrocyte',
                'fetal-thymus', 'naive-cd4', 'total-cd4', 'activated-total-cd4',
                'non-activated-total-cd4', 'naive-cd8', 'total-cd8',
                'naive-b', 'total-b']

# listing bedpe columns with pc-hic columns 
bedpe_cols = ['baitChr', 'baitStart', 'baitEnd',
                 'oeChr', 'oeStart', 'oeEnd']

In [4]:
# save interactions
for cell, new_cell in zip(cells, new_cells): 
    cell_bedpe = main_bedpe.loc[:, bedpe_cols + [cell]]
    cell_bedpe.iloc[:, 0] = 'chr' + cell_bedpe.iloc[:, 0].astype(str)
    cell_bedpe.iloc[:, 3] = 'chr' + cell_bedpe.iloc[:, 3].astype(str)
    bedpe_fn = os.path.join(outdir, '{}.bedpe'.format(new_cell))
    cell_bedpe.to_csv(bedpe_fn, sep='\t', header=False, index=False)